In [135]:
import pandas as pd
import numpy as np
from pprint import pprint

In [136]:
# Importando dados
movies_df = pd.read_csv('../filmes.csv')

In [137]:
# Visualizando colunas do dataset
variables = list(movies_df.columns.values)
variables_str = '\n'.join(sorted(variables))
print('Variáveis:\n{}'.format(variables_str))
movies_df.head()
len(movies_df.columns)

Variáveis:
actor_1_facebook_likes
actor_1_name
actor_2_facebook_likes
actor_2_name
actor_3_facebook_likes
actor_3_name
aspect_ratio
budget
cast_total_facebook_likes
color
content_rating
country
director_facebook_likes
director_name
duration
facenumber_in_poster
genres
gross
imdb_score
language
movie_facebook_likes
movie_imdb_link
movie_title
num_critic_for_reviews
num_user_for_reviews
num_voted_users
plot_keywords
title_year


28

## 1. Deletando entradas com 1/3 dos valores ausentes

In [138]:
from math import ceil

num_variables = len(variables)
threshold = ceil((1/3)*num_variables)
print("Número de variáveis: {}. Limite de {} valores ausentes".format(num_variables, threshold))

    
# 1. Substituir 0's por NaN's
movies_df = movies_df.applymap(lambda x: np.nan if x == 0.0 else x)
size = len(movies_df)
# 2. Contabilizar NaNs para cada entrada
count_missing = movies_df.isnull().sum(axis='columns')
# 3. Remover
movies_df = movies_df[count_missing < threshold]
print("{} de {} entradas removidas".format(size-len(movies_df), size))

Número de variáveis: 28. Limite de 10 valores ausentes
16 de 5043 entradas removidas


## 2.0 Separando variáveis categóricas das variáveis numéricas

In [139]:
categorical_variables = ['actor_1_name', 'actor_2_name', 'actor_3_name', 'country', 'director_name', 'genres', 'language', 'movie_title', 'movie_imdb_link', 'plot_keywords', 'color', 'content_rating']
numerical_variables = [v for v in variables if v not in categorical_variables]

categorical_df = movies_df[categorical_variables]
numerical_df = movies_df[numerical_variables]

## 3. Processando variáveis categóricas

In [140]:
# Descrição do dataset categórico puro
categorical_df.describe()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
count,5025,5023,5016,5024,4928,5027,5018,5027,5027,4881,5011,4739
unique,2086,3026,3518,64,2387,911,47,4901,4903,4751,2,18
top,Robert De Niro,Morgan Freeman,Ben Mendelsohn,USA,Steven Spielberg,Drama,English,Victor Frankenstein,http://www.imdb.com/title/tt2638144/?ref_=fn_t...,based on novel,Color,R
freq,49,20,8,3796,26,236,4693,3,3,4,4802,2118


In [141]:
# Amostra do dataset
categorical_df.head()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
0,CCH Pounder,Joel David Moore,Wes Studi,USA,James Cameron,Action|Adventure|Fantasy|Sci-Fi,English,Avatar,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,avatar|future|marine|native|paraplegic,Color,PG-13
1,Johnny Depp,Orlando Bloom,Jack Davenport,USA,Gore Verbinski,Action|Adventure|Fantasy,English,Pirates of the Caribbean: At World's End,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,goddess|marriage ceremony|marriage proposal|pi...,Color,PG-13
2,Christoph Waltz,Rory Kinnear,Stephanie Sigman,UK,Sam Mendes,Action|Adventure|Thriller,English,Spectre,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,bomb|espionage|sequel|spy|terrorist,Color,PG-13
3,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,USA,Christopher Nolan,Action|Thriller,English,The Dark Knight Rises,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,deception|imprisonment|lawlessness|police offi...,Color,PG-13
5,Daryl Sabara,Samantha Morton,Polly Walker,USA,Andrew Stanton,Action|Adventure|Sci-Fi,English,John Carter,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,alien|american civil war|male nipple|mars|prin...,Color,PG-13


### 3.0 Uniformizando o texto

In [142]:
# Tirando todos os espaços em branco antes e depois das palavras
categorical_df = categorical_df.applymap(lambda x: x.strip() if type(x) is str else x)

### 3.1 Tratando valores ausentes
Estratégias:
1. Nomes dos atores: Deixar ausente
2. País: 
3. Diretor
4. Idioma:
5. Título: deixar ausente
6. Keywords: deixar ausente
8. Link do imdb: deixar ausente
9. Cor: colorido se número de likes no fb != nan

#### Atores

In [143]:
missing = categorical_df[categorical_df['actor_1_name'].isna() & categorical_df['actor_2_name'].isna() | categorical_df['actor_3_name'].isna()]
missing.describe()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
count,9,7,0.0,11,11,11,11,11,11,9,11,8
unique,9,7,0.0,7,11,7,2,11,11,9,1,5
top,Billy West,Bobby Kendall,NaN,USA,James Bidgood,Documentary,English,Sisters in Law,http://www.imdb.com/title/tt0497116/?ref_=fn_t...,eastern philosophy|healing|india|life,Color,Not Rated
freq,1,1,NaN,5,1,5,10,1,1,1,11,3


#### País

In [144]:
country_df = categorical_df.country;
country_df.describe()

count     5024
unique      64
top        USA
freq      3796
Name: country, dtype: object

In [145]:
print("Tabela de Frequências")
country_df.value_counts().head()

Tabela de Frequências


USA        3796
UK          447
France      154
Canada      126
Germany      97
Name: country, dtype: int64

In [146]:
## Valores Ausentes
categorical_df[categorical_df['country'].isna()]

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
2370,Nicole 'Snooki' Polizzi,Jenni 'Jwoww' Farley,Paul 'Pauly D' DelVecchio,NaN,NaN,Comedy|Drama|Reality-TV|Romance,English,"Gone, Baby, Gone",http://www.imdb.com/title/tt1697237/?ref_=fn_t...,lifestyle,Color,TV-14
3397,Dominic Cooper,Joseph Gilgun,Ruth Negga,NaN,NaN,Adventure|Drama|Fantasy|Mystery,English,Preacher,http://www.imdb.com/title/tt5016504/?ref_=fn_t...,heaven and hell|preacher|supernatural|vampire|...,Color,TV-MA
4021,Chris Brochu,Jeff Fahey,Rita Wilson,NaN,Daniel Petrie Jr.,Drama|Thriller,English,Dawn Patrol,http://www.imdb.com/title/tt2073661/?ref_=fn_t...,desert|held at gunpoint|marine|revenge|sex on ...,Color,NaN


In [147]:
# Ajustando todos valores ausentes manualmente, baseando-se no país
categorical_df.loc[2370, 'country'] = "USA"
categorical_df.loc[2370, 'director_name'] = "Ben Affleck"

categorical_df.loc[3397, 'country'] = "USA"
categorical_df.loc[3397, 'director_name'] = "Sam Catlin"

categorical_df.loc[4021, 'country'] = "USA"

#### Diretor

In [148]:
categorical_df.loc[categorical_df['director_name'].isna()].describe()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
count,97,97,97,97,0.0,97,97,97,97,89,96,64
unique,93,95,94,9,0.0,61,5,95,95,87,2,7
top,Krystyna Janda,Olaf Lubaszenko,Bokeem Woodbine,USA,NaN,Comedy,English,Dekalog,http://www.imdb.com/title/tt0092337/?ref_=fn_t...,meaning of life|moral challenge|morality|searc...,Color,TV-14
freq,2,2,2,71,NaN,7,91,2,2,2,93,26


In [149]:
# Estratégia adotada: descartar
categorical_df = categorical_df.loc[~categorical_df['director_name'].isna()]

#### Gêneros

In [151]:
categorical_df['genres'].describe()

count      4930
unique      900
top       Drama
freq        232
Name: genres, dtype: object

In [154]:
categorical_df[categorical_df['genres'].isna()].describe()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
count,0,0,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,0,0


#### Keywords

In [156]:
categorical_df['plot_keywords'].describe()

count               4792
unique              4664
top       based on novel
freq                   4
Name: plot_keywords, dtype: object

In [157]:
categorical_df[categorical_df['plot_keywords'].isna()].describe() #Estratégia: deixar sem

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
count,138,137,136,138,138,138,136,138,138,0.0,134,71
unique,135,135,135,21,135,71,16,137,137,0.0,2,7
top,Demi Moore,Landon Liboiron,Michael Wincott,USA,Jon Cassar,Drama,English,Forsaken,http://www.imdb.com/title/tt2271563/?ref_=fn_t...,NaN,Color,R
freq,2,2,2,82,2,18,112,2,2,NaN,131,30


#### Idioma

In [117]:
categorical_df['language'].value_counts().head()

English     4693
French        73
Spanish       40
Hindi         28
Mandarin      26
Name: language, dtype: int64

In [118]:
categorical_df.loc[categorical_df['language'].isna()]

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
3086,Jon Gries,Taylor Handley,Trent Ford,USA,Christopher Cain,Drama|History|Romance|Western,NaN,September Dawn,http://www.imdb.com/title/tt0473700/?ref_=fn_t...,massacre|mormon|settler|utah|wagon train,Color,R
3539,Debi Derryberry,Kate Higgins,Cindy Robinson,USA,Richard Rich,Action|Adventure|Animation|Comedy|Drama|Family...,NaN,Alpha and Omega 4: The Legend of the Saw Tooth...,http://www.imdb.com/title/tt4061848/?ref_=fn_t...,blindness|cave|spirit|wolf|wolf cub,NaN,NaN
3869,Sid Caesar,Dom DeLuise,Bernadette Peters,USA,Mel Brooks,Comedy|Romance,NaN,Silent Movie,http://www.imdb.com/title/tt0075222/?ref_=fn_t...,black comedy|friend|modern silent movie|silent...,Color,PG
4110,William Morgan Sheppard,Kevin Gage,Brianna Brown,USA,Michael Landon Jr.,Drama|Family|Western,NaN,Love's Abiding Joy,http://www.imdb.com/title/tt0785025/?ref_=fn_t...,19th century|faith|mayor|ranch|sheriff,Color,PG
4409,Matthew Ziff,T.J. Storm,Sam Medina,USA,John Stockwell,Action,NaN,Kickboxer: Vengeance,http://www.imdb.com/title/tt3082898/?ref_=fn_t...,NaN,NaN,NaN
4630,Justin Baldoni,Luke Perry,Leonor Varela,USA,Jonathan Meyers,Drama,NaN,A Fine Step,http://www.imdb.com/title/tt1604100/?ref_=fn_t...,NaN,NaN,PG
4810,Lillian Gish,Mae Marsh,Walter Long,USA,D.W. Griffith,Drama|History|War,NaN,Intolerance: Love's Struggle Throughout the Ages,http://www.imdb.com/title/tt0006864/?ref_=fn_t...,huguenot|intolerance|medicis|protestant|wedding,Black and White,Not Rated
4885,John Gilbert,Renée Adorée,Claire Adams,USA,King Vidor,Drama|Romance|War,NaN,The Big Parade,http://www.imdb.com/title/tt0015624/?ref_=fn_t...,chewing gum|climbing a tree|france|translation...,Black and White,Not Rated
4958,Stephen Carr,Johnnie Walker,Mary Carr,USA,Harry F. Millarde,Crime|Drama,NaN,Over the Hill to the Poorhouse,http://www.imdb.com/title/tt0011549/?ref_=fn_t...,family relationships|gang|idler|poorhouse|thief,Black and White,NaN


In [119]:
#Todos os que faltam são americanos, logo são em inglês
categorical_df.loc[categorical_df['language'].isna(), 'language'] = 'English'

#### Classificação Indicativa

In [158]:
categorical_df.loc[categorical_df['content_rating'].isna()].describe()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating
count,254,253,252,255,255,255,252,255,255,188,252,0.0
unique,243,249,249,31,246,124,26,253,253,186,2,0.0
top,Christopher Judge,Shin'ya Tsukamoto,Riteish Deshmukh,USA,Jon Knautz,Drama,English,Godzilla Resurgence,http://www.imdb.com/title/tt0796314/?ref_=fn_t...,blood|godzilla|monster|sequel,Color,NaN
freq,2,2,2,128,2,21,172,2,2,2,238,NaN


In [159]:
# Estratégia: Deixar como está

#### Cor

In [163]:
# Visualização dos dados antes de tratar dados ausentes
categorical_df['color'].astype(str).value_counts()

Color              4709
Black and White     206
nan                  15
Name: color, dtype: int64

In [164]:
#Estratégia: remover as que faltam, dado que são poucas
categorical_df = categorical_df[~categorical_df['color'].isna()]

### 3.2 Codificando colunas simples

In [126]:
from sklearn.feature_extraction.text import CountVectorizer
import string

def remove_special_chars(x, exceptions = ['|']):
    x = x.replace(' ', '').lower()
    for char in string.punctuation:
        if char not in exceptions:
            x = x.replace(char, '')
    return x
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

#### Atores

In [127]:
columns_actors = ['actor_1_name', 'actor_2_name', 'actor_3_name']
categorical_df[columns_actors] = categorical_df[columns_actors].astype(str).applymap(remove_special_chars)
actors = categorical_df[columns_actors].values.flatten()

In [128]:
actor_vectorizer = CountVectorizer()
actor_vectorizer.fit(actors)
vocabulary = sorted(actor_vectorizer.vocabulary_)

In [129]:
join_actors = categorical_df[columns_actors[0]].astype(str) +'|' +  categorical_df[columns_actors[1]].astype(str) + '|' +categorical_df[columns_actors[2]].astype(str)
categorical_df.drop(columns = columns_actors, inplace=True)
join_actors.head()

0            cchpounder|joeldavidmoore|wesstudi
1         johnnydepp|orlandobloom|jackdavenport
2    christophwaltz|rorykinnear|stephaniesigman
3     tomhardy|christianbale|josephgordonlevitt
5        darylsabara|samanthamorton|pollywalker
dtype: object

In [130]:
#actors_vect =  actor_vectorizer.transform(categorical_df['actors']).toarray()
#categorical_df['actors'] = actors_vect
vect_actors = actor_vectorizer.transform(join_actors.values)
actors_df =pd.DataFrame(vect_actors.toarray())
actors_df.rename(lambda x: 'actor_'+vocabulary[int(x)], axis='columns', inplace=True)
categorical_df = categorical_df.join(actors_df)
categorical_df.head()

,country,director_name,genres,language,movie_title,movie_imdb_link,plot_keywords,color,content_rating,actor_50cent,...,actor_zoëbell,actor_zoëkravitz,actor_zoëpoledouris,actor_zubaidasahar,actor_zuhairhaddad,actor_álexangulo,actor_ángelamolina,actor_émiliedequenne,actor_ólafurdarriólafsson,actor_óscarjaenada
0,USA,James Cameron,Action|Adventure|Fantasy|Sci-Fi,English,Avatar,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,avatar|future|marine|native|paraplegic,Color,PG-13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,USA,Gore Verbinski,Action|Adventure|Fantasy,English,Pirates of the Caribbean: At World's End,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,goddess|marriage ceremony|marriage proposal|pi...,Color,PG-13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,UK,Sam Mendes,Action|Adventure|Thriller,English,Spectre,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,bomb|espionage|sequel|spy|terrorist,Color,PG-13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,USA,Christopher Nolan,Action|Thriller,English,The Dark Knight Rises,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,deception|imprisonment|lawlessness|police offi...,Color,PG-13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,USA,Andrew Stanton,Action|Adventure|Sci-Fi,English,John Carter,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,alien|american civil war|male nipple|mars|prin...,Color,PG-13,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### País, diretor, idioma, Título, Cor, Classificação Indicativa, Título e Link

In [100]:
categorical_df.drop(columns=['movie_title', 'movie_imdb_link'], inplace=True) # não importam
variables = ['country', 'director_name', 'language', 'color', 'content_rating']
dummies = pd.get_dummies(categorical_df[variables])
categorical_df.drop(columns=variables, inplace=True)
categorical_df = categorical_df.join(dummies)
categorical_df.head()

,genres,plot_keywords,actor_50cent,actor_aaliyah,actor_aaronashmore,actor_aaronhill,actor_aaronhughes,actor_aaronkwok,actor_aaronstanford,actor_aaronstaton,...,content_rating_Passed,content_rating_R,content_rating_TV-14,content_rating_TV-G,content_rating_TV-MA,content_rating_TV-PG,content_rating_TV-Y,content_rating_TV-Y7,content_rating_Unrated,content_rating_X
0,Action|Adventure|Fantasy|Sci-Fi,avatar|future|marine|native|paraplegic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Action|Adventure|Fantasy,goddess|marriage ceremony|marriage proposal|pi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Action|Adventure|Thriller,bomb|espionage|sequel|spy|terrorist,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,Action|Thriller,deception|imprisonment|lawlessness|police offi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5,Action|Adventure|Sci-Fi,alien|american civil war|male nipple|mars|prin...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### 3.3 Codificando colunas em formato de lista (gênero e keywords)

#### Gêneros

In [99]:
#Processando gênero
# Transformando string em lista
genres =  categorical_df.genres.apply(remove_special_chars)
genres.head()

0    Action|Adventure|Fantasy|SciFi
1          Action|Adventure|Fantasy
2         Action|Adventure|Thriller
3                   Action|Thriller
5            Action|Adventure|SciFi
Name: genres, dtype: object

In [26]:
vocabulary = set()
genres.apply(lambda x: vocabulary.update(x.split('|')))

genre_vectorizer = CountVectorizer()
genre_vectorizer.fit(vocabulary)
vocabulary = sorted(genre_vectorizer.vocabulary_)
vocabulary

['action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'filmnoir',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'news',
 'realitytv',
 'romance',
 'scifi',
 'short',
 'sport',
 'thriller',
 'war',
 'western']

In [27]:
vect_genres = genre_vectorizer.transform(genres.values)
genres_binary_df = pd.DataFrame(vect_genres.toarray())
genres_binary_df.rename(lambda x: 'genre_'+vocabulary[int(x)], axis='columns', inplace=True)
genres_binary_df.head()

,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_family,genre_fantasy,...,genre_mystery,genre_news,genre_realitytv,genre_romance,genre_scifi,genre_short,genre_sport,genre_thriller,genre_war,genre_western
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [28]:
# Colocando de volta no Dataframe
categorical_df.drop('genres', axis='columns', inplace=True)
categorical_df = categorical_df.join(genres_binary_df)
categorical_df.head()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,language,movie_title,movie_imdb_link,plot_keywords,color,...,genre_mystery,genre_news,genre_realitytv,genre_romance,genre_scifi,genre_short,genre_sport,genre_thriller,genre_war,genre_western
0,CCH Pounder,Joel David Moore,Wes Studi,USA,James Cameron,English,Avatar,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,avatar|future|marine|native|paraplegic,Color,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Johnny Depp,Orlando Bloom,Jack Davenport,USA,Gore Verbinski,English,Pirates of the Caribbean: At World's End,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,goddess|marriage ceremony|marriage proposal|pi...,Color,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Christoph Waltz,Rory Kinnear,Stephanie Sigman,UK,Sam Mendes,English,Spectre,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,bomb|espionage|sequel|spy|terrorist,Color,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,USA,Christopher Nolan,English,The Dark Knight Rises,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,deception|imprisonment|lawlessness|police offi...,Color,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,Daryl Sabara,Samantha Morton,Polly Walker,USA,Andrew Stanton,English,John Carter,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,alien|american civil war|male nipple|mars|prin...,Color,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Keywords

In [29]:
#Lembrete do problema de '1000000 b.c.
keywords = categorical_df.plot_keywords.astype(str).apply(lambda x: x.split('|'))
all_keywords = set()
keywords.apply(lambda x: all_keywords.update(x))
sorted(all_keywords)

['10 year old',
 '1000000 b.c.',
 '1190s',
 '12 step program',
 '12 year old',
 '12 year time span',
 '12th century',
 '13 year old',
 '13 year olds',
 '13th birthday',
 '14 year old',
 '14th century',
 '15 year old',
 '1520s',
 '15th birthday',
 '15th century',
 '16 year old',
 '16th century',
 '1770s',
 '17th century',
 '18 wheeler',
 '1800s',
 '1810s',
 '1830s',
 '1850s',
 '1860s',
 '1880s',
 '1890s',
 '18th birthday',
 '18th century',
 '1910s',
 '1920s',
 '1930s',
 '1940s',
 '1950s',
 '1955 chevrolet',
 '1959 cadillac',
 '1960s',
 '1969 dodge charger',
 '1970s',
 '1980s',
 '1988 winter olympics',
 '1990s',
 '19th century',
 '1st century',
 '1st century b.c.',
 '20 years later',
 '2000s',
 '2010s',
 '2020s',
 '2030s',
 '20th century',
 '21 year old',
 '21st birthday',
 '21st century',
 '22 year old',
 '22nd century',
 '23 year time span',
 '23rd century',
 '27th century',
 '2nd century',
 '3 dimensional',
 '30 year old',
 '35 mm digital camera',
 '3d',
 '3d in title',
 '40 year old'

In [30]:
# Processando keywords
keywords = categorical_df.plot_keywords.astype(str).apply(remove_special_chars)
all_keywords = set()
keywords.apply(lambda x: all_keywords.update(x.split('|')))
len(all_keywords)

8063

In [31]:
keyword_vectorizer = CountVectorizer()
keyword_vectorizer.fit(all_keywords)
vocabulary = sorted(keyword_vectorizer.vocabulary_)
len(vocabulary)

8091

In [32]:
vect_keywords = genre_vectorizer.transform(keywords.values)
keywords_binary_df = pd.DataFrame(vect_keywords.toarray())
keywords_binary_df.rename(lambda x: 'keyword_'+vocabulary[int(x)], axis='columns', inplace=True)
keywords_binary_df.head()

,keyword_1000000bc,keyword_10yearold,keyword_1190s,keyword_12stepprogram,keyword_12thcentury,keyword_12yearold,keyword_12yeartimespan,keyword_13thbirthday,keyword_13yearold,keyword_13yearolds,...,keyword_15yearold,keyword_16thcentury,keyword_16yearold,keyword_1770s,keyword_17thcentury,keyword_1800s,keyword_1810s,keyword_1830s,keyword_1850s,keyword_1860s
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Colocando de volta no Dataframe
categorical_df.drop('plot_keywords', axis='columns', inplace=True)
categorical_df = categorical_df.join(keywords_binary_df)
categorical_df.head()

,actor_1_name,actor_2_name,actor_3_name,country,director_name,language,movie_title,movie_imdb_link,color,content_rating,...,keyword_15yearold,keyword_16thcentury,keyword_16yearold,keyword_1770s,keyword_17thcentury,keyword_1800s,keyword_1810s,keyword_1830s,keyword_1850s,keyword_1860s
0,CCH Pounder,Joel David Moore,Wes Studi,USA,James Cameron,English,Avatar,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,Color,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Johnny Depp,Orlando Bloom,Jack Davenport,USA,Gore Verbinski,English,Pirates of the Caribbean: At World's End,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,Color,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Christoph Waltz,Rory Kinnear,Stephanie Sigman,UK,Sam Mendes,English,Spectre,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,Color,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,USA,Christopher Nolan,English,The Dark Knight Rises,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,Color,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Daryl Sabara,Samantha Morton,Polly Walker,USA,Andrew Stanton,English,John Carter,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,Color,PG-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
